# Convolutional Neural Network for Sentiment Classification

This notebook comprises the network architecture used for senciment classification using as input human faces.

#### Import Dependencies

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, Flatten, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

#### Hyper-Parameters

In [ ]:
num_classes = 7

image_size = (48, 48)
input_shape = (48, 48, 1)

labels = {0:'angry',1:'disgust',2:'fear',3:'happy', 4:'sad',5:'surprise',6:'neutral'}

batch_size = 128
num_epochs = 5
validation_split = .1
test_split = .1
patience = 3
dropout = .2

dataset_path = '../../dataset/fer2013.csv'
model_output = '../models/best_model_weights.hdf5'

#### Load Dataset

In [ ]:
data = pd.read_csv(dataset_path)

In [ ]:
def load_dataset():
    pixels = data['pixels']
    
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(48, 48)
        faces.append(face.astype('float32'))

    faces = np.asarray(faces)
    emotions = to_categorical(data['emotion'], num_classes)
    
    return faces, emotions

#### Pre-Process Dataset

In [ ]:
faces, emotions = load_dataset()

train_faces, val_faces, train_emotions, val_emotions = train_test_split(faces, emotions, test_size=validation_split)
train_faces, test_faces, train_emotions, test_emotions = train_test_split(train_faces, train_emotions, test_size=test_split)

train_faces = train_faces.reshape(-1, 48, 48, 1)
val_faces = val_faces.reshape(-1, 48, 48, 1)
test_faces = test_faces.reshape(-1, 48, 48, 1)

train_faces = train_faces.astype("float32")/255.
val_faces = val_faces.astype("float32")/255.
test_faces = test_faces.astype("float32")/255.

#### Network Architecture

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(7, 7), padding='same', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(filters=16, kernel_size=(7, 7), activation='relu', padding='same'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(dropout))

model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding='same'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(dropout))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(dropout))

model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(dropout))

model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=num_classes, kernel_size=(3, 3), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(dropout))

model.add(Dense(num_classes, activation = "softmax"))

model.summary()


#### Define Callbacks

In [ ]:
modelCheckpoint = ModelCheckpoint(monitor='val_accuracy', filepath=model_output,
                                               save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=patience)


if not os.path.exists('../models'):
    os.makedirs('../models')

tensorboard = TensorBoard("logs/mood-classification-I")

#### Data Augmnetation

In [ ]:
train_datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

test_datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

train_generator = train_datagen.flow(train_faces, train_emotions, batch_size = batch_size)

validation_generator = test_datagen.flow(val_faces, val_emotions, batch_size = batch_size)

#### Compile Model

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# history = model.fit_generator(train_generator, epochs = num_epochs, verbose = 1,
#           validation_data = train_generator, callbacks=[modelCheckpoint, earlyStopping, tensorboard])

In [ ]:
history = model.fit(train_faces, train_emotions, batch_size = 128, epochs = num_epochs, verbose = 1,
          validation_data = (val_faces, val_emotions),
          callbacks=[modelCheckpoint, earlyStopping, tensorboard])

#### Predict

In [ ]:
saved_model = load_model(model_output)
predictions = saved_model.predict_classes(test_faces, verbose = 2)
print(predictions)
# np.std(history.history['loss'])

#### Test a Single face

In [ ]:
reshaped_face = np.reshape(test_faces[239], (-1, 48, 48, 1))
face = saved_model.predict_classes(reshaped_face, verbose = 2)
print(labels[face[0]])

#### Plot Image

In [ ]:
plt.imshow(test_faces[239].reshape(48,48))

#### Final Accuracy

In [ ]:
final_loss, final_acc = saved_model.evaluate(test_faces, test_emotions, verbose = 2)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))